<a href="https://colab.research.google.com/github/SofiSiro/Skempi/blob/main/Skempi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [202]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [203]:
# Загрузка скачанной базы данных skempi
df = pd.read_csv('/content/skempi_v2.csv', sep=';')
df

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,koff_mut_parsed,koff_wt (s^(-1)),koff_wt_parsed,dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7080,3QIB_ABP_CD,KP9R,KP8R,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",2.4E-04,2.400000e-04,5.5E-06,5.500000e-06,...,0.500,2.2E-02,0.022,NaN,NaN,NaN,NaN,NaN,SPR,2
7081,3QIB_ABP_CD,TP12A,TP11A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPR,2
7082,3QIB_ABP_CD,TP12S,TP11S,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",3.38E-05,3.380000e-05,5.5E-06,5.500000e-06,...,0.134,2.2E-02,0.022,NaN,NaN,NaN,NaN,NaN,SPR,2
7083,3QIB_ABP_CD,TP12N,TP11N,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",4.34E-05,4.340000e-05,5.5E-06,5.500000e-06,...,0.175,2.2E-02,0.022,NaN,NaN,NaN,NaN,NaN,SPR,2


In [204]:
# Смотрим структуру дф, типы объектов, количество пропущенных значений
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   #Pdb                          7085 non-null   object 
 1   Mutation(s)_PDB               7085 non-null   object 
 2   Mutation(s)_cleaned           7085 non-null   object 
 3   iMutation_Location(s)         7085 non-null   object 
 4   Hold_out_type                 3311 non-null   object 
 5   Hold_out_proteins             7085 non-null   object 
 6   Affinity_mut (M)              7085 non-null   object 
 7   Affinity_mut_parsed           6800 non-null   float64
 8   Affinity_wt (M)               7085 non-null   object 
 9   Affinity_wt_parsed            7083 non-null   float64
 10  Reference                     7085 non-null   object 
 11  Protein 1                     7085 non-null   object 
 12  Protein 2                     7085 non-null   object 
 13  Tem

In [205]:
df.columns

Index(['#Pdb', 'Mutation(s)_PDB', 'Mutation(s)_cleaned',
       'iMutation_Location(s)', 'Hold_out_type', 'Hold_out_proteins',
       'Affinity_mut (M)', 'Affinity_mut_parsed', 'Affinity_wt (M)',
       'Affinity_wt_parsed', 'Reference', 'Protein 1', 'Protein 2',
       'Temperature', 'kon_mut (M^(-1)s^(-1))', 'kon_mut_parsed',
       'kon_wt (M^(-1)s^(-1))', 'kon_wt_parsed', 'koff_mut (s^(-1))',
       'koff_mut_parsed', 'koff_wt (s^(-1))', 'koff_wt_parsed',
       'dH_mut (kcal mol^(-1))', 'dH_wt (kcal mol^(-1))',
       'dS_mut (cal mol^(-1) K^(-1))', 'dS_wt (cal mol^(-1) K^(-1))', 'Notes',
       'Method', 'SKEMPI version'],
      dtype='object')

In [206]:
# Удаляем ненужные колонки: не несущие важную для предсказательной модели информацию,
# колонки с большим количество пропусков, колонки типа object, которые в не будут преведены к int или float.
df = df.drop(['Mutation(s)_PDB', 'iMutation_Location(s)', 'Hold_out_type', 'Hold_out_proteins',
              'Affinity_mut (M)', 'Affinity_wt (M)', 'Reference', 'Protein 1', 'Protein 2',
              'kon_mut (M^(-1)s^(-1))', 'kon_mut_parsed',
              'kon_wt (M^(-1)s^(-1))', 'kon_wt_parsed', 'koff_mut (s^(-1))',
              'koff_mut_parsed', 'koff_wt (s^(-1))', 'koff_wt_parsed',
              'dH_mut (kcal mol^(-1))', 'dH_wt (kcal mol^(-1))',
              'dS_mut (cal mol^(-1) K^(-1))', 'dS_wt (cal mol^(-1) K^(-1))', 'Notes',
              'Method', 'SKEMPI version'], axis=1)
df

,#Pdb,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature
0,1CSE_E_I,LI38G,5.260000e-11,1.120000e-12,294
1,1CSE_E_I,LI38S,8.330000e-12,1.120000e-12,294
2,1CSE_E_I,LI38P,1.020000e-07,1.120000e-12,294
3,1CSE_E_I,LI38I,1.720000e-10,1.120000e-12,294
4,1CSE_E_I,LI38D,1.920000e-09,1.120000e-12,294
...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,2.400000e-04,5.500000e-06,298
7081,3QIB_ABP_CD,TP11A,1.100000e-03,5.500000e-06,298
7082,3QIB_ABP_CD,TP11S,3.380000e-05,5.500000e-06,298
7083,3QIB_ABP_CD,TP11N,4.340000e-05,5.500000e-06,298


In [207]:
# Убираем nan значения
df = df.dropna(subset=["Affinity_mut_parsed", "Affinity_wt_parsed"])
df

,#Pdb,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature
0,1CSE_E_I,LI38G,5.260000e-11,1.120000e-12,294
1,1CSE_E_I,LI38S,8.330000e-12,1.120000e-12,294
2,1CSE_E_I,LI38P,1.020000e-07,1.120000e-12,294
3,1CSE_E_I,LI38I,1.720000e-10,1.120000e-12,294
4,1CSE_E_I,LI38D,1.920000e-09,1.120000e-12,294
...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,2.400000e-04,5.500000e-06,298
7081,3QIB_ABP_CD,TP11A,1.100000e-03,5.500000e-06,298
7082,3QIB_ABP_CD,TP11S,3.380000e-05,5.500000e-06,298
7083,3QIB_ABP_CD,TP11N,4.340000e-05,5.500000e-06,298


In [208]:
# Проверяем колонку температуры, т.к. эти данные понадобятся для расчетов
df['Temperature'].unique()

array(['294', '298(assumed)', '298', '293', '295', '305', '310', '323',
       '273', '299', '296', '303', '300', '277', '283', '308', '333',
       '291', '279', '301', '281', '288', '285', nan, '338', '318'],
      dtype=object)

In [209]:
# Заменям значения содержащие текст и заполняем пропуски
# Приводим к типу int
df['Temperature'] = df['Temperature'].replace('298(assumed)', '298')
df['Temperature'] = df['Temperature'].fillna(298)
df['Temperature'] = df['Temperature'].astype(int)

<ipython-input-209-571581410397>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Temperature'] = df['Temperature'].replace('298(assumed)', '298')
<ipython-input-209-571581410397>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Temperature'] = df['Temperature'].fillna(298)
<ipython-input-209-571581410397>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [210]:
# Проверяем на наличие неподходящих значений
df['Temperature'].unique()

array([294, 298, 293, 295, 305, 310, 323, 273, 299, 296, 303, 300, 277,
       283, 308, 333, 291, 279, 301, 281, 288, 285, 338, 318])

In [211]:
# Рассчитываем ddG по формуле представленой на сайте skempi
r = 8.314/4184

df['delta_wt'] = r * df['Temperature'] * np.log(df["Affinity_wt_parsed"])
df['delta_mut'] = r * df['Temperature'] * np.log(df["Affinity_mut_parsed"])
df['ddG'] = df['delta_mut'] - df['delta_wt']

<ipython-input-211-a6e1571f4f12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_wt'] = r * df['Temperature'] * np.log(df["Affinity_wt_parsed"])
<ipython-input-211-a6e1571f4f12>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_mut'] = r * df['Temperature'] * np.log(df["Affinity_mut_parsed"])
<ipython-input-211-a6e1571f4f12>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [212]:
# Добавляем классификацию по знаку ddG
df["label"] = (df["ddG"] > 0).astype(int)

<ipython-input-212-9199085b1271>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = (df["ddG"] > 0).astype(int)


In [213]:
df

,#Pdb,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature,delta_wt,delta_mut,ddG,label
0,1CSE_E_I,LI38G,5.260000e-11,1.120000e-12,294,-16.075989,-13.827155,2.248833,1
1,1CSE_E_I,LI38S,8.330000e-12,1.120000e-12,294,-16.075989,-14.903760,1.172229,1
2,1CSE_E_I,LI38P,1.020000e-07,1.120000e-12,294,-16.075989,-9.404712,6.671276,1
3,1CSE_E_I,LI38I,1.720000e-10,1.120000e-12,294,-16.075989,-13.135001,2.940988,1
4,1CSE_E_I,LI38D,1.920000e-09,1.120000e-12,294,-16.075989,-11.725555,4.350434,1
...,...,...,...,...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,2.400000e-04,5.500000e-06,298,-7.171435,-4.935527,2.235909,1
7081,3QIB_ABP_CD,TP11A,1.100000e-03,5.500000e-06,298,-7.171435,-4.034016,3.137419,1
7082,3QIB_ABP_CD,TP11S,3.380000e-05,5.500000e-06,298,-7.171435,-6.096254,1.075181,1
7083,3QIB_ABP_CD,TP11N,4.340000e-05,5.500000e-06,298,-7.171435,-5.948216,1.223219,1


In [214]:
# Разделим строку, чтобы извлечь информацию о мутации
def mut_name(mutation):
    match = re.match(r'([A-Z])(\d+)([A-Z])', str(mutation))
    if match:
        wt, pos, mut = match.groups()
        return wt, mut
    return None, None

In [215]:
df[['wt_aa', 'mut_aa']] = df['Mutation(s)_cleaned'].apply(lambda x: pd.Series(mut_name(x)))

<ipython-input-215-be33f76bec47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['wt_aa', 'mut_aa']] = df['Mutation(s)_cleaned'].apply(lambda x: pd.Series(mut_name(x)))
<ipython-input-215-be33f76bec47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['wt_aa', 'mut_aa']] = df['Mutation(s)_cleaned'].apply(lambda x: pd.Series(mut_name(x)))


In [216]:
df

,#Pdb,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature,delta_wt,delta_mut,ddG,label,wt_aa,mut_aa
0,1CSE_E_I,LI38G,5.260000e-11,1.120000e-12,294,-16.075989,-13.827155,2.248833,1,None,None
1,1CSE_E_I,LI38S,8.330000e-12,1.120000e-12,294,-16.075989,-14.903760,1.172229,1,None,None
2,1CSE_E_I,LI38P,1.020000e-07,1.120000e-12,294,-16.075989,-9.404712,6.671276,1,None,None
3,1CSE_E_I,LI38I,1.720000e-10,1.120000e-12,294,-16.075989,-13.135001,2.940988,1,None,None
4,1CSE_E_I,LI38D,1.920000e-09,1.120000e-12,294,-16.075989,-11.725555,4.350434,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,2.400000e-04,5.500000e-06,298,-7.171435,-4.935527,2.235909,1,None,None
7081,3QIB_ABP_CD,TP11A,1.100000e-03,5.500000e-06,298,-7.171435,-4.034016,3.137419,1,None,None
7082,3QIB_ABP_CD,TP11S,3.380000e-05,5.500000e-06,298,-7.171435,-6.096254,1.075181,1,None,None
7083,3QIB_ABP_CD,TP11N,4.340000e-05,5.500000e-06,298,-7.171435,-5.948216,1.223219,1,None,None


In [217]:
# Переводим информацию о мутации из тестового формата в числовой, чтобы применить в качестве признака при обучении модели
aa_list = list('ACDEFGHIKLMNPQRSTVWY')
for aa in aa_list:
    df[f"wt_{aa}"] = (df["wt_aa"] == aa).astype(int)
    df[f"mut_{aa}"] = (df["mut_aa"] == aa).astype(int)

<ipython-input-217-acb29caf4204>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"wt_{aa}"] = (df["wt_aa"] == aa).astype(int)
<ipython-input-217-acb29caf4204>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"mut_{aa}"] = (df["mut_aa"] == aa).astype(int)
<ipython-input-217-acb29caf4204>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [218]:
df

,#Pdb,Mutation(s)_cleaned,Affinity_mut_parsed,Affinity_wt_parsed,Temperature,delta_wt,delta_mut,ddG,label,wt_aa,...,wt_S,mut_S,wt_T,mut_T,wt_V,mut_V,wt_W,mut_W,wt_Y,mut_Y
0,1CSE_E_I,LI38G,5.260000e-11,1.120000e-12,294,-16.075989,-13.827155,2.248833,1,None,...,0,0,0,0,0,0,0,0,0,0
1,1CSE_E_I,LI38S,8.330000e-12,1.120000e-12,294,-16.075989,-14.903760,1.172229,1,None,...,0,0,0,0,0,0,0,0,0,0
2,1CSE_E_I,LI38P,1.020000e-07,1.120000e-12,294,-16.075989,-9.404712,6.671276,1,None,...,0,0,0,0,0,0,0,0,0,0
3,1CSE_E_I,LI38I,1.720000e-10,1.120000e-12,294,-16.075989,-13.135001,2.940988,1,None,...,0,0,0,0,0,0,0,0,0,0
4,1CSE_E_I,LI38D,1.920000e-09,1.120000e-12,294,-16.075989,-11.725555,4.350434,1,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,2.400000e-04,5.500000e-06,298,-7.171435,-4.935527,2.235909,1,None,...,0,0,0,0,0,0,0,0,0,0
7081,3QIB_ABP_CD,TP11A,1.100000e-03,5.500000e-06,298,-7.171435,-4.034016,3.137419,1,None,...,0,0,0,0,0,0,0,0,0,0
7082,3QIB_ABP_CD,TP11S,3.380000e-05,5.500000e-06,298,-7.171435,-6.096254,1.075181,1,None,...,0,0,0,0,0,0,0,0,0,0
7083,3QIB_ABP_CD,TP11N,4.340000e-05,5.500000e-06,298,-7.171435,-5.948216,1.223219,1,None,...,0,0,0,0,0,0,0,0,0,0


In [219]:
# Проверяем на наличие нечисловых признаков в дф
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6798 entries, 0 to 7084
Data columns (total 51 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #Pdb                 6798 non-null   object 
 1   Mutation(s)_cleaned  6798 non-null   object 
 2   Affinity_mut_parsed  6798 non-null   float64
 3   Affinity_wt_parsed   6798 non-null   float64
 4   Temperature          6798 non-null   int64  
 5   delta_wt             6798 non-null   float64
 6   delta_mut            6798 non-null   float64
 7   ddG                  6798 non-null   float64
 8   label                6798 non-null   int64  
 9   wt_aa                0 non-null      object 
 10  mut_aa               0 non-null      object 
 11  wt_A                 6798 non-null   int64  
 12  mut_A                6798 non-null   int64  
 13  wt_C                 6798 non-null   int64  
 14  mut_C                6798 non-null   int64  
 15  wt_D                 6798 non-null   int64 

In [220]:
# Удаляем нечисловые признаки
df = df.drop(['wt_aa', 'mut_aa'], axis=1)

In [222]:
# Переносим колонку предикта на 1 место
df = df[['label'] + [col for col in df.columns if col != 'label']]

In [223]:
# Делим дф на признаки и предикты
x = df.iloc[:, 3::] # Не захватываем #Pdb и Mutation(s)_cleaned, т.к. они нечисловые, но нужны для индентификации белков и мутаций
y = df.iloc[:,0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

In [224]:
# Обучаем простую модель логической регресии
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [225]:
# Предсказываем признаки тестовой выборки
y_pred = model.predict(x_test)

In [226]:
# Оцениваем точность модели
accuracy_score(y_test, y_pred)

0.9963235294117647

In [227]:
confusion_matrix(y_test, y_pred)

array([[ 331,    0],
       [   5, 1024]])